## Learning Game and Public Engagement

The simple game implemented in the provided code is an engaging interactive experience designed to help users locate the Large Magellanic Cloud (LMC) within the sky, utilizing Textual MOC data structures as the core element of the gameplay. Textual MOCs combine spatial and textual data, allowing the game to provide both visual and informational feedback as the user interacts with different regions of the sky. When the game begins, users navigate the Aladin Lite viewer, searching for the correct region associated with the LMC. The goal is to hover the cursor over the magenta-colored MOC, which is dynamically generated based on the Textual MOC data. Upon successfully identifying the correct region, a popup displays a message confirming the discovery, and the user's score is updated. The game also includes auditory feedback, with sounds playing to signal whether the user has found the correct or incorrect region.

The code demonstrates an application of Textual MOCs in an interactive educational setting, showing how spatial and textual information can be combined to create a rich user experience. 

In [1]:
import os
import http.server
import socketserver
import webbrowser
from threading import Thread
import json

# Define the server port
PORT = 8000

# HTML content with updated paths to CSS and JS files inside the 'aladin_game_d2' directory
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Large Magellanic Cloud Hunt</title>
    <link rel="stylesheet" href="css/styles.css">
    <script src="https://aladin.cds.unistra.fr/AladinLite/api/v3/latest/aladin.js" charset="utf-8"></script>
</head>
<body>
    <!-- Game instructions -->
    <h2>Hunting for Celestial Objects</h2>
    <div id="instructions">
        Your task is to find the Large Magellanic Cloud (LMC) within the sky among the objects indicated 
        by the magenta-colored MOCs. Press the "Start Game" button to begin playing.
        If you have difficulty finding it, use the search button at the top to navigate directly to the LMC's position.
    </div>
    
    <div id="aladin-lite-div">
        <div id="popup">test play with textual moc</div>
    </div>
    <div id="controls">
        <button id="start-game">Start Game</button>
    </div>
    <div id="game-status">Press "Start Game" to begin</div>
    <div id="score">Score: 0</div>
    <div id="timer">Time remaining: <span id="time">30</span>s</div>
    <div id="countdown"></div>
    <audio id="success-sound" preload="auto">
        <source src="https://cdn.jsdelivr.net/gh/ggreco77/TextualMOC@main/AladinGame/sounds/won.mp3" type="audio/mpeg">
        Your browser does not support the audio element.
    </audio>
    <audio id="error-sound" preload="auto">
        <source src="https://cdn.jsdelivr.net/gh/ggreco77/TextualMOC@main/AladinGame/sounds/lost.mp3" type="audio/mpeg">
        Your browser does not support the audio element.
    </audio>
    <script src="js/script.js"></script>
</body>
</html>
"""

# CSS content
css_content = """
body {
    background: linear-gradient(to right, #00c6ff, #0072ff);
    font-family: Arial, sans-serif;
    color: #fff;
    display: flex;
    flex-direction: column;
    align-items: center;
    justify-content: center;
    min-height: 100vh;
    margin: 0;
    padding: 20px;
}

#aladin-lite-div {
    width: 100%;
    max-width: 500px;
    height: 500px;
    margin-bottom: 20px;
    position: relative;
}
#popup {
    position: absolute;
    padding: 10px;
    background-color: rgba(0, 0, 0, 0.7);
    color: white;
    border-radius: 5px;
    display: none;
    z-index: 10;
    pointer-events: none;
    white-space: nowrap;
}
#controls {
    display: flex;
    justify-content: center;
    margin-bottom: 20px;
}
#start-game {
    padding: 10px 20px;
    background-color: #007bff;
    color: white;
    border: none;
    border-radius: 5px;
    cursor: pointer;
}
#score, #timer, #game-status, #instructions {
    text-align: center;
    font-size: 20px;
    margin: 10px 0;
}
#timer {
    transition: color 0.5s ease;
}
#countdown {
    font-size: 60px;
    color: rgb(250, 249, 247);
    display: none;
}
"""

# JavaScript content
javascript_content = r"""
let aladin; // Aladin Lite instance
let cursorInMOC = false; // Flag to check if the cursor is in a MOC region
let gameStarted = false; // Flag to track if the game has started
let score = 0; // Player's score
let timeLeft = 30; // Game duration in seconds
let timerInterval; // Timer interval
const popup = document.getElementById('popup'); // Popup element for displaying messages
const aladinContainer = document.getElementById('aladin-lite-div'); // Container for the Aladin Lite viewer
const successSound = document.getElementById('success-sound'); // Sound for successful action
const errorSound = document.getElementById('error-sound'); // Sound for unsuccessful action
const startGameButton = document.getElementById('start-game'); // Start game button
const scoreDisplay = document.getElementById('score'); // Score display element
const timerDisplay = document.getElementById('timer'); // Timer display element
const gameStatusDisplay = document.getElementById('game-status'); // Game status display element
const countdownDisplay = document.getElementById('countdown'); // Countdown display element
const timeElement = document.getElementById('time'); // Time display element

successSound.volume = 1; // Set maximum volume for success sound
errorSound.volume = 1; // Set maximum volume for error sound

// Function to start the game
function startGame() {
    if (gameStarted) return;
    gameStarted = true;
    score = 0;
    timeLeft = 30;
    scoreDisplay.textContent = `Score: ${score}`;
    timerDisplay.style.color = 'white';
    timeElement.textContent = `${timeLeft}`;
    countdownDisplay.style.display = 'none'; // Hide countdown at the start
    gameStatusDisplay.textContent = "Game in progress...";
    startGameButton.disabled = true;

    // Start the timer
    timerInterval = setInterval(() => {
        timeLeft--;
        timeElement.textContent = timeLeft; // Update the element with the remaining time

        if (timeLeft <= 5) {
            timerDisplay.style.color = ''; // Change color of the timer
            countdownDisplay.textContent = timeLeft;
            countdownDisplay.style.display = 'block'; // Show the countdown in large font
        } else {
            timerDisplay.style.color = 'white';
            countdownDisplay.style.display = 'none'; // Hide the countdown
        }

        if (timeLeft <= 0) {
            endGame();
        }
    }, 1000);
}

// Function to end the game
function endGame(message = `Time's up! Final score: ${score}`) {
    clearInterval(timerInterval);
    gameStarted = false;
    startGameButton.disabled = false;
    gameStatusDisplay.textContent = message;
    countdownDisplay.style.display = 'none'; // Hide the countdown
}

let mocList = [];

// Load MOC data from mocDataList.json
fetch('moc/mocDataList.json')
.then(response => response.json())
.then(mocDataList => {

    A.init.then(() => {
        aladin = A.aladin('#aladin-lite-div', { target: 'Sgr', fov: 180 });

        // Create MOC at the same position in the sky
        mocList = mocDataList.map(moc_with_text => {
            const mocData = processMOCWithText(moc_with_text);
            const moc = A.MOCFromJSON(mocData, { opacity: 0.45, color: 'magenta', lineWidth: 1, fill: true, perimeter: true });
            aladin.addMOC(moc);
            return { moc, text: moc_with_text.text, isTarget: moc_with_text.isTarget };
        });

        // Handle mouse movement events
        aladin.on('mouseMove', function (pos) {
            if (!pos || !gameStarted) return;

            let found = false;
            mocList.forEach(item => {
                const isInMOC = item.moc.contains(pos.ra, pos.dec);

                if (isInMOC) {
                    found = true;
                    if (item.isTarget && !cursorInMOC) {
                        score += 10; // Only for the winning MOC
                        scoreDisplay.textContent = `Score: ${score}`;
                        popup.textContent = item.text;
                        popup.style.display = 'block'; // Show the popup
                        if (successSound) {
                            successSound.currentTime = 0;
                            successSound.play().catch(error => console.log("Error playing success sound: ", error));
                        }
                        endGame("Congratulations! You found the Large Magellanic Cloud!");
                    } else if (!item.isTarget && !cursorInMOC) {
                        popup.textContent = item.text; // Error message for non-winning MOC
                        popup.style.display = 'block';
                        if (errorSound) {
                            errorSound.currentTime = 0;
                            errorSound.play().catch(error => console.log("Error playing error sound: ", error));
                        }
                    }
                    cursorInMOC = true;
                }
            });

            if (!found && cursorInMOC) {
                cursorInMOC = false;
                popup.style.display = 'none'; // Hide the popup
            }
        });

        // Update the position of the popup based on mouse movement
        aladinContainer.addEventListener('mousemove', function (event) {
            if (cursorInMOC) {
                const rect = aladinContainer.getBoundingClientRect();
                popup.style.left = `${event.clientX - rect.left + 10}px`;
                popup.style.top = `${event.clientY - rect.top + 10}px`;
            }
        });
    });

    startGameButton.addEventListener('click', startGame);
});

// Function to process MOC with text
function processMOCWithText(mocWithText) {
    const mocData = {};
    for (const key in mocWithText) {
        if (key !== 'text' && key !== 'isTarget') {
            mocData[key] = mocWithText[key];
        }
    }
    return mocData;
}
"""

# MOC data (saved as JSON)
moc_data_list = [
    {
        "5": [8263, 8274],
        "6": [33047, 33049, 33050, 33051, 33072, 33073, 33076, 33077, 33078, 33090, 33091, 33094, 33095, 33100, 33101, 33102, 33106, 33120, 33121],
        "7": [132186, 132187, 132195, 132245, 132297, 132300, 132301, 132303, 132316, 132317, 132318, 132354, 132355, 132358, 132359, 132370, 132371, 132374, 132375, 132412, 132413, 132414, 132418, 132448, 132449, 132450, 132488, 132489, 132496, 132497],
        "8": [528631, 528637, 528638, 528639, 528699, 528701, 528702, 528703, 528731, 528734, 528735, 528739, 528741, 528742, 528743, 528775, 528779, 528988, 528989, 528991, 529184, 529185, 529187, 529196, 529197, 529208, 529209, 529211, 529276, 529277, 529278, 529426, 529427, 529430, 529431, 529474, 529475, 529478, 529479, 529490, 529491, 529494, 529495, 529660, 529661, 529662, 529666, 529676, 529678, 529679, 529712, 529714, 529720, 529722, 529804, 529805, 529806, 529808, 529824, 529825, 529826, 529960, 529961, 529968, 529969, 529970, 529972, 529992, 529993, 529994, 529996, 530000, 530001, 530002],
        "9": [2114517, 2114519, 2114523, 2114545, 2114547, 2114783, 2114795, 2114801, 2114802, 2114803, 2114922, 2114923, 2114931, 2114934, 2114935, 2114951, 2114953, 2114954, 2114955, 2114961, 2114962, 2114963, 2115093, 2115094, 2115095, 2115097, 2115098, 2115099, 2115109, 2115110, 2115111, 2115113, 2115114, 2115115, 2115961, 2115963, 2116052, 2116053, 2116055, 2116744, 2116745, 2116747, 2116772, 2116773, 2116796, 2116797, 2116840, 2116841, 2116843, 2117116, 2117117, 2117204, 2117205, 2117376, 2117377, 2117380, 2117381, 2117392, 2117393, 2117396, 2117658, 2117659, 2117662, 2117663, 2117718, 2117719, 2117890, 2117891, 2117894, 2117895, 2117906, 2117907, 2117910, 2117911, 2117954, 2117955, 2117958, 2117959, 2117970, 2117971, 2117974, 2118652, 2118653, 2118654, 2118668, 2118670, 2118671, 2118682, 2118708, 2118710, 2118711, 2118760, 2118762, 2118763, 2118854, 2118860, 2118862, 2118884, 2118886, 2118892, 2118894, 2119228, 2119230, 2119240, 2119241, 2119242, 2119264, 2119308, 2119309, 2119310, 2119312, 2119313, 2119314, 2119328, 2119329, 2119330, 2119848, 2119849, 2119856, 2119857, 2119858, 2119860, 2119861, 2119884, 2119885, 2119886, 2119892, 2119893, 2119894, 2119896, 2119980, 2119981, 2119982, 2119988, 2119989, 2119992, 2120012, 2120016, 2120017, 2120064],
        "10": [8457725, 8457727, 8458067, 8458073, 8458074, 8458075, 8458085, 8458087, 8458173, 8458175, 8458177, 8458179, 8458185, 8458186, 8458187, 8459130, 8459131, 8459163, 8459165, 8459166, 8459167, 8459174, 8459175, 8459177, 8459178, 8459179, 8459203, 8459515, 8459517, 8459518, 8459519, 8459679, 8459686, 8459687, 8459722, 8459723, 8459731, 8459734, 8459735, 8459802, 8459803, 8459810, 8459811, 8459842, 8459843, 8460349, 8460350, 8460351, 8460366, 8460367, 8460371, 8460385, 8460386, 8460387, 8460429, 8460430, 8460431, 8460433, 8460434, 8460435, 8460449, 8460450, 8460451, 8463637, 8463639, 8463645, 8463647, 8463669, 8463671, 8463677, 8463679, 8463765, 8463840, 8463841, 8463843, 8463849, 8464197, 8464199, 8464216, 8464217, 8464219, 8464244, 8464245, 8464247, 8466984, 8466985, 8466987, 8467076, 8467077, 8467097, 8467100, 8467101, 8467103, 8467169, 8467172, 8467173, 8467175, 8467196, 8467197, 8467368, 8467369, 8467371, 8468472, 8468473, 8468476, 8468477, 8468753, 8468756, 8468757, 8468800, 8468801, 8468804, 8468805, 8469588, 8469589, 8469590, 8469760, 8469761, 8469764, 8469765, 8469776, 8469777, 8469780, 8470586, 8470587, 8470590, 8470591, 8470642, 8470643, 8470646, 8470647, 8470811, 8470814, 8470815, 8470858, 8470859, 8470862, 8470863, 8471570, 8471571, 8471574, 8471575, 8471618, 8471619, 8471622, 8471623, 8471634, 8471635, 8471638, 8471639, 8471810, 8471811, 8471814, 8471815, 8471826, 8471827, 8471830, 8471831, 8471900, 8471902, 8471903, 8474620, 8474634, 8474635, 8474638, 8474678, 8474679, 8474722, 8474732, 8474734, 8474735, 8474746, 8474838, 8474839, 8475016, 8475018, 8475019, 8475044, 8475046, 8475066, 8475408, 8475410, 8475411, 8475444, 8475446, 8475452, 8475454, 8475540, 8475542, 8475548, 8475550, 8475572, 8476916, 8476917, 8476918, 8476944, 8476945, 8476946, 8476952, 8476954, 8476972, 8476974, 8477064, 8477244, 8477245, 8477246, 8477260, 8477264, 8477280, 8477282, 8477324, 8477325, 8477326, 8477328, 8477329, 8477330, 8477344, 8477345, 8479400, 8479408, 8479409, 8479412, 8479413, 8479436, 8479437, 8479438, 8479448, 8479449, 8479548, 8479549, 8479580, 8479581, 8479582, 8479588, 8479589, 8479590, 8479600, 8479616, 8479617, 8479618, 8479620, 8479932, 8479933, 8479934, 8479960, 8479961, 8479962, 8479964, 8479972, 8479973, 8480052, 8480053, 8480056, 8480072, 8480073, 8480074, 8480080, 8480128, 8480129, 8480130, 8480260, 8480261, 8480262, 8480264, 8480272],
        "text": "You found the Large Magellanic Cloud! +10 Points",
        "isTarget": True
    },
    {
        "6": [33293, 33295, 33304, 33306, 33317],
        "7": [133151, 133177, 133179, 133194, 133265, 133267, 133273, 133275, 133276, 133277, 133278, 133312, 133314],
        "8": [532599, 532603, 532679, 532685, 532686, 532687, 532713, 532715, 532770, 532771, 532780, 532782, 532783, 532882, 532888, 532890, 533057, 533059, 533065, 533066, 533067, 533089, 533091, 533097, 533116, 533189, 533200, 533252, 533254, 533280],
        "9": [2130394, 2130395, 2130405, 2130406, 2130407, 2130409, 2130410, 2130411, 2130711, 2130737, 2130739, 2130821, 2130823, 2130829, 2130830, 2130831, 2130859, 2131074, 2131075, 2131078, 2131096, 2131098, 2131099, 2131124, 2131126, 2131127, 2131178, 2131520, 2131522, 2131648, 2131649, 2131650, 2131656, 2131658, 2131680, 2131682, 2132225, 2132227, 2132233, 2132235, 2132257, 2132258, 2132259, 2132353, 2132355, 2132361, 2132363, 2132385, 2132396, 2132397, 2132399, 2132468, 2132469, 2132470, 2132472, 2132473, 2132474, 2132752, 2132753, 2132755, 2132804, 2132805, 2133012, 2133013, 2133014, 2133020, 2133040, 2133041, 2133042, 2133048, 2133124, 2133126, 2133128, 2133129, 2133130],
        "10": [8521471, 8521535, 8521563, 8521566, 8521567, 8521573, 8521574, 8521575, 8521615, 8521619, 8521635, 8522837, 8522838, 8522839, 8522841, 8522843, 8522861, 8522862, 8522863, 8522941, 8522943, 8522947, 8522953, 8522954, 8522955, 8523291, 8523313, 8523314, 8523315, 8523399, 8523405, 8523407, 8523429, 8523430, 8523431, 8524316, 8524318, 8524319, 8524362, 8524388, 8524390, 8524391, 8524408, 8524410, 8524411, 8524500, 8524502, 8524503, 8524682, 8524704, 8524706, 8524707, 8526084, 8526086, 8526092, 8526094, 8526604, 8526606, 8526628, 8526630, 8526636, 8526638, 8526752, 8526754, 8526760, 8528899, 8528905, 8528907, 8528929, 8528931, 8528937, 8528939, 8529025, 8529027, 8529408, 
        8529409, 8529411, 8529417, 8529419, 8529441, 8529443, 8529449, 8529451, 8529548, 8529549, 8529551, 8529573, 8529592, 8529593, 8529595, 8529884, 8529885, 8529886, 8529900, 8529901, 8529902, 8529904, 8529905, 8529906, 8530964, 8530965, 8530967, 8531016, 8531017, 8531057, 8531060, 8531061, 8531224, 8531225, 8531264, 8532060, 8532062, 8532088, 8532089, 8532090, 8532172, 8532173, 8532174, 8532176, 8532196, 8532200, 8532202, 8532500, 8532501, 8532502, 8532508, 8532524, 8532525, 8532526, 8532528, 8532530, 8532608, 8532609, 8532610],
        "text": "Not the right one, try again!",
        "isTarget": False
    },
    {
        "6": [2623, 2708, 2709],
        "7": [10487, 10489, 10490, 10491, 10664, 10666, 10667, 10823, 10840, 10841, 11008],
        "8": [41919, 41935, 41945, 41946, 41947, 41955, 42632, 42634, 42635, 42638, 42660, 42662, 42663, 43285, 43287, 43317, 43319, 43369, 43376, 43377, 43378, 43380, 44036, 44040],
        "9": [167671, 167735, 167738, 167739, 167771, 167774, 167775, 167778, 167779, 167813, 167814, 167815, 167817, 167818, 167819, 170506, 170532, 170534, 170535, 170546, 170644, 170646, 170698, 170720, 170722, 170728, 173137, 173139, 173145, 173147, 173265, 173267, 173273, 173301, 173472, 173473, 173475, 173488, 173489, 173492, 173516, 173517, 173524, 176148, 176149, 176150, 176152, 176153, 176154, 176164, 176165, 176166, 176168, 176169, 176170, 176176],
        "10": [670677, 670678, 670679, 670683, 670701, 670703, 670911, 670939, 670949, 670950, 670951, 671035, 671038, 671039, 671081, 671082, 671083, 671106, 671107, 671109, 671110, 671111, 671247, 671251, 671267, 682030, 682031, 682042, 682132, 682134, 682135, 682178, 682190, 682226, 682232, 682234, 682235, 682588, 682590, 682591, 682784, 682786, 682787, 682886, 682920, 682922, 692555, 692577, 692579, 692585, 692586, 692587, 693056, 693057, 693059, 693065, 693067, 693101, 693200, 693201, 693213],
        "text": "Not the right one, try again!",
        "isTarget": False
    }
]

# Define the base directory as 'aladin_game_d2'
base_dir = 'aladin_game_d2'

# Create necessary directories if they don't exist
os.makedirs(os.path.join(base_dir, 'css'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'js'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'moc'), exist_ok=True)

# Write files into the 'game' directory
with open(os.path.join(base_dir, 'index.html'), 'w') as f:
    f.write(html_content)

with open(os.path.join(base_dir, 'css', 'styles.css'), 'w') as f:
    f.write(css_content)

with open(os.path.join(base_dir, 'js', 'script.js'), 'w') as f:
    f.write(javascript_content)

with open(os.path.join(base_dir, 'moc', 'mocDataList.json'), 'w') as f:
    json.dump(moc_data_list, f)

# Function to start the server
def start_server():
    # Change the directory to 'aladin_game_d2' where the HTML file is located
    os.chdir(os.path.abspath(base_dir))

    # Handler to serve files
    Handler = http.server.SimpleHTTPRequestHandler

    # Start the server
    with socketserver.TCPServer(("", PORT), Handler) as httpd:
        # Open the page in the default web browser
        webbrowser.open(f"http://localhost:{PORT}/index.html")
        print(f"Serving at port {PORT}")
        httpd.serve_forever()

# Run the server in a separate thread to avoid blocking
server_thread = Thread(target=start_server)
server_thread.start()

Serving at port 8000


127.0.0.1 - - [17/Sep/2025 19:30:49] "GET /index.html HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2025 19:30:49] "GET /css/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2025 19:30:49] "GET /js/script.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2025 19:30:50] "GET /moc/mocDataList.json HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2025 19:31:00] code 404, message File not found
127.0.0.1 - - [17/Sep/2025 19:31:00] "GET /favicon.ico HTTP/1.1" 404 -
